In [5]:

import sympy as sp
from scipy.stats import ortho_group

# Skew-symmetric matrix function for cross product
def skew(vec):
    """Returns the skew-symmetric matrix [vec]_x for a 3x1 vector."""
    return sp.Matrix([
        [0, -vec[2], vec[1]],
        [vec[2], 0, -vec[0]],
        [-vec[1], vec[0], 0]
    ])


# Generate a random 3x3 orthogonal matrix
random_orthogonal_matrix = ortho_group.rvs(3)
random_orthogonal_matrix_2 = ortho_group.rvs(3)

print(random_orthogonal_matrix)

[[-0.22686615  0.66560337  0.71098797]
 [-0.94694874  0.01988031 -0.32076916]
 [ 0.2276397   0.74604083 -0.62578211]]


In [6]:
import numpy as np
from scipy.spatial.transform import Rotation as R
from scipy.linalg import expm, logm


def h_acc(state, a_n, omega_n):
    R_imu, v_imu, p_imu, b_omega, b_a, R_c, p_c = state[:3], state[3:6].reshape(3, 1), \
        state[6:9].reshape(3, 1), state[9:12], state[12:15], state[15:18], state[18:21]

    R_c = expm(skew(state[15:18].reshape(3,))) @ random_orthogonal_matrix
    R_imu = expm(skew(state[:3].reshape(3,))) @ random_orthogonal_matrix_2
    
    v_c = R_c.T @ (( R_imu.T @ v_imu).flatten() + np.cross(omega_n - b_omega, p_c))
    b_vector = (( R_imu.T @ v_imu).flatten() + np.cross(omega_n - b_omega, p_c)).reshape(3,)
    # print("the B matrix phuc derivation is \n", (R_c.T @ skew (b_vector)).T)
    # print("the B_our matrix is \n", ((R_c.T @ skew(R_imu.T @ v_c))).T)
    # print("the R_imu.T is \n", R_imu.T)
    # print("the matrix C is \n", -skew(omega_n - b_omega))
    # print("FOR BIAS OMEGA \n", R_c.T @ skew(p_c))

    obs_paper = v_c
    return obs_paper

def compute_jacobian_paper(x_0, a_n, omega_n, d=1e-9):
    # Define small perturbations
    n = len(x_0)
    jacobian = np.zeros((3, n))

    # Compute the observation at the nominal state
    obs_nominal = h_acc(x_0, a_n, omega_n)

    for i in range(n):
        # Compute the observation with perturbed state
        x_0_perturbed = np.copy(x_0)
        x_0_perturbed[i] += d
        obs_perturbed = h_acc(x_0_perturbed, a_n, omega_n)
        # Calculate Jacobian element by finite difference
        # print(obs_perturbed)
        jacobian[:, i] = (obs_perturbed - obs_nominal) / d

    return jacobian

def h_acc_new(state, a_n, omega_n):
    R_imu, v_imu, p_imu, b_omega, b_a, R_c, p_c = state[:3], state[3:6].reshape(3, 1), \
        state[6:9].reshape(3, 1), state[9:12], state[12:15], state[15:18], state[18:21]

    R_c = expm(skew(state[15:18].reshape(3,))) @ random_orthogonal_matrix
    R_imu = expm(skew(state[:3].reshape(3,))) @ random_orthogonal_matrix_2
    
    v_c = R_c.T @ (( R_imu.T @ v_imu).flatten() + np.cross(omega_n - b_omega, p_c))
    b_vector = (( R_imu.T @ v_imu).flatten() + np.cross(omega_n - b_omega, p_c)).reshape(3,)
    omega_car = R_c.T @(omega_n - b_omega)
    a_car = R_c.T @ (a_n - b_a + np.cross(omega_n - b_omega, np.cross(omega_n - b_omega, p_c)))
    # print("the B matrix phuc derivation is \n", (R_c.T @ skew (b_vector)).T)
    # print("the B_our matrix is \n", ((R_c.T @ skew(R_imu.T @ v_c))).T)
    # print("the R_imu.T is \n", R_imu.T)
    # print("the matrix C is \n", -skew(omega_n - b_omega))
    # print("FOR BIAS OMEGA \n", R_c.T @ skew(p_c))

    ############## find analytical solution #################
    # for SE2(3) term:
    obs_new = a_car[1] - v_c[0] * omega_car[2]
    return obs_new

def compute_jacobian_new_obser(x_0, a_n, omega_n, d=1e-9):
    # Define small perturbations
    n = len(x_0)
    jacobian = np.zeros((n))

    # Compute the observation at the nominal state
    obs_nominal = h_acc_new(x_0, a_n, omega_n)

    for i in range(n):
        # Compute the observation with perturbed state
        x_0_perturbed = np.copy(x_0)
        x_0_perturbed[i] += d
        obs_perturbed = h_acc_new(x_0_perturbed, a_n, omega_n)
        # Calculate Jacobian element by finite difference
        # print(obs_perturbed)
        jacobian[i] = (obs_perturbed - obs_nominal) / d

    return jacobian

# Example usage:
chi_imu = np.array([0.0, 0.0, 0.0])
v_imu = np.array([0.5, 1.0, 3.0])
p_imu = np.array([0.3, 0.1, 0.8])
b_a = np.array([0.0, 0.0, 0.0])
b_omega = np.array([0.0, 0.0, 0.0])
chi_c = np.array([0.0, 0.0, 0.0])
p_c = np.array([0.1, 0.1, 0.1])

# Nominal state

a_n = np.array([1.0, 1.0, 1.0])
omega_n = np.array([0.5, 0.1, 1.0])
x_0 = np.concatenate((chi_imu, v_imu, p_imu, b_omega, b_a, chi_c, p_c))

In [7]:
compute_jacobian_paper(x_0, a_n, omega_n).T

array([[-1.38597267, -2.76995338,  0.63752792],
       [-0.15143575,  0.92239882,  2.8941749 ],
       [ 0.28147262,  0.15419266, -1.07097953],
       [ 0.21169733, -0.31858072, -0.92395347],
       [-0.13955104, -0.94554697,  0.29405206],
       [ 0.96732089, -0.06668754,  0.24462882],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [-0.11745893, -0.07261614,  0.03050121],
       [ 0.04545031,  0.00804379, -0.13367701],
       [ 0.07200818,  0.06457235,  0.10317569],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 1.25521371,  2.13119011, -1.5946286 ],
       [-0.41873305, -0.47788107,  1.20653215],
       [-0.49092108, -1.88867255, -2.43020715],
       [-0.9697132 , -0.05472378, -0.25819097],
       [ 0.3406857 , -0.29258285, -1.02387882],
       [ 0.45078741,  0.05662026,  0.231

In [8]:
from sympy import symbols, Matrix, cos, sin

# Define symbolic vectors for SE2(3)
chi_r = Matrix(symbols(r'chi_{r1} chi_{r2} chi_{r3}')).reshape(3, 1)
chi_v = Matrix(symbols(r'chi_{v1} chi_{v2} chi_{v3}')).reshape(3, 1)
chi_p = Matrix(symbols(r'chi_{p1} chi_{p2} chi_{p3}')).reshape(3, 1)

# Define the elements in the SE2(3) matrix
R = Matrix(3, 3, symbols('R1:10'))  # 3x3 rotation matrix
v = Matrix(3, 1, symbols('v1:4'))  # 3x1 velocity vector
p = Matrix(3, 1, symbols('p1:4'))  # 3x1 position vector

# Compute the inverse SE2(3) transformation
chi_inverse = Matrix.hstack(R.T, -R.T @ v, -R.T @ p)

# Define the full SE2(3) matrix
chi_epsilon = Matrix.vstack(chi_r, chi_v, chi_p)

# Define angle as the norm of the rotation vector (first 3 elements of state)
angle = (chi_r.norm())  # Use norm of chi_r for rotation
s = sin(angle)
c = cos(angle)

# Calculate a_ and b_
a_ = (1 - c) / angle
b_ = (angle - s / (angle**3))

# Corrected f_error expression
f_error = (R.T @ chi_v +
           a_ * R.T @ (skew(chi_r) @ chi_v) +
           b_ * R.T @ (skew(chi_r) @ (skew(chi_r) @ chi_v)))

# Jacobian of f_error w.r.t. chi_epsilon
Jacobian = f_error.jacobian(chi_epsilon)

# Correct substitution: Set all elements of chi_v and chi_p to 0
subs_dict = {var: 0.00 for var in chi_v}  # Set all chi_v elements to 0
subs_dict.update({var: 0.00 for var in chi_p})  # Set all chi_p elements to 0

# Apply substitution and simplify
Jacobian_simplified = Jacobian.subs(subs_dict)

# Print the final Jacobian
Jacobian_simplified

Matrix([
[0, 0, 0, R1*(-chi_{r2}**2 - chi_{r3}**2)*(sqrt(Abs(chi_{r1})**2 + Abs(chi_{r2})**2 + Abs(chi_{r3})**2) - sin(sqrt(Abs(chi_{r1})**2 + Abs(chi_{r2})**2 + Abs(chi_{r3})**2))/(Abs(chi_{r1})**2 + Abs(chi_{r2})**2 + Abs(chi_{r3})**2)**(3/2)) + R1 + R4*chi_{r1}*chi_{r2}*(sqrt(Abs(chi_{r1})**2 + Abs(chi_{r2})**2 + Abs(chi_{r3})**2) - sin(sqrt(Abs(chi_{r1})**2 + Abs(chi_{r2})**2 + Abs(chi_{r3})**2))/(Abs(chi_{r1})**2 + Abs(chi_{r2})**2 + Abs(chi_{r3})**2)**(3/2)) + R4*chi_{r3}*(1 - cos(sqrt(Abs(chi_{r1})**2 + Abs(chi_{r2})**2 + Abs(chi_{r3})**2)))/sqrt(Abs(chi_{r1})**2 + Abs(chi_{r2})**2 + Abs(chi_{r3})**2) + R7*chi_{r1}*chi_{r3}*(sqrt(Abs(chi_{r1})**2 + Abs(chi_{r2})**2 + Abs(chi_{r3})**2) - sin(sqrt(Abs(chi_{r1})**2 + Abs(chi_{r2})**2 + Abs(chi_{r3})**2))/(Abs(chi_{r1})**2 + Abs(chi_{r2})**2 + Abs(chi_{r3})**2)**(3/2)) - R7*chi_{r2}*(1 - cos(sqrt(Abs(chi_{r1})**2 + Abs(chi_{r2})**2 + Abs(chi_{r3})**2)))/sqrt(Abs(chi_{r1})**2 + Abs(chi_{r2})**2 + Abs(chi_{r3})**2), R1*chi_{r1}*chi_{r